In [6]:
import pandas as pd
import numpy as np
import re

In [379]:
import os
path = "주가총액csv"
ju_file_list = os.listdir(path)
ju_file_list

['2010.csv',
 '2011.csv',
 '2012.csv',
 '2013.csv',
 '2014.csv',
 '2015.csv',
 '2016.csv',
 '2017.csv',
 '2018.csv',
 '2019.csv',
 '2020.csv']

In [389]:
csv_df_li = []
for f in ju_file_list:
    df = pd.read_csv(f"./주가총액csv/{f}", encoding='euc-kr')
    year = f.replace('.csv', '')
    df['년도'] = year
    csv_df_li.append(df)

df = pd.concat(csv_df_li)
df = df.sort_values(['종목명', '년도'], ascending=False)
companys = df['종목명'].unique()
fi_df_dic = {}
for co in companys:
    fi_df = df[df['종목명'] == co][['종가','시가총액', '상장주식수']].T
    fi_df.columns = df[df['종목명'] == co]['년도'].values
    fi_df_dic[co] = fi_df

In [407]:
fi_df_dic

{'힘스인터내셔널':               2014         2013
 종가           13000         6270
 시가총액   20800000000  10032000000
 상장주식수      1600000      1600000,
 '힘스':                2020          2019         2018          2017
 종가            12800         27200         9480         23950
 시가총액   144796620800  153846409600  53619998640  125932692500
 상장주식수      11312236       5656118      5656118       5258150,
 '히스토스템':               2010
 종가            1100
 시가총액   31894109500
 상장주식수     28994645,
 '히든챔피언스팩1호':               2012         2011         2010
 종가            2020         1860         2005
 시가총액   32522000000  29946000000  32280500000
 상장주식수     16100000     16100000     16100000,
 '희림':               2020         2019         2018         2017         2016  \
 종가            3905         4285         4625         4645         5160   
 시가총액   54367264875  59657805375  64391446875  64669896375  71839971000   
 상장주식수     13922475     13922475     13922475     13922475     13922475   
 
     

In [459]:
path = "./fsdata"
file_list = os.listdir(path)
file_list

['3S.xlsx',
 'AJ네트웍스.xlsx',
 'AK홀딩스.xlsx',
 'APS홀딩스.xlsx',
 'AP시스템.xlsx',
 'BGF.xlsx',
 'BGF리테일.xlsx',
 'BNGT.xlsx',
 'BNK금융지주.xlsx',
 'BYC.xlsx',
 'CJ CGV.xlsx',
 'CJ ENM.xlsx',
 'CJ 바이오사이언스.xlsx',
 'CJ.xlsx',
 'CJ대한통운.xlsx',
 'CJ씨푸드.xlsx',
 'CJ제일제당.xlsx',
 'CJ프레시웨이.xlsx',
 'CMG제약.xlsx',
 'CS홀딩스.xlsx',
 'DB.xlsx',
 'DB하이텍.xlsx',
 'DL.xlsx',
 'DMS.xlsx',
 'DRB동일.xlsx',
 'DSR.xlsx',
 'E1.xlsx',
 'EDGC.xlsx',
 'EG.xlsx',
 'EMB.xlsx',
 'EMW.xlsx',
 'ESR켄달스퀘어리츠.xlsx',
 'ES큐브.xlsx',
 'FSN.xlsx',
 'GH신소재.xlsx',
 'GS.xlsx',
 'GS건설.xlsx',
 'GS글로벌.xlsx',
 'GS리테일.xlsx',
 'HB솔루션.xlsx',
 'HB테크놀러지.xlsx',
 'HDC.xlsx',
 'HDC랩스.xlsx',
 'HDC현대EP.xlsx',
 'HDC현대산업개발.xlsx',
 'HJ중공업.xlsx',
 'HLB.xlsx',
 'HLB생명과학.xlsx',
 'HLB제약.xlsx',
 'HLB테라퓨틱스.xlsx',
 'HMM.xlsx',
 'HRS.xlsx',
 'HSD엔진.xlsx',
 'iMBC.xlsx',
 'ISC.xlsx',
 'ITX-AI.xlsx',
 'JTC.xlsx',
 'JW생명과학.xlsx',
 'JW신약.xlsx',
 'JW중외제약.xlsx',
 'JW홀딩스.xlsx',
 'JYP Ent..xlsx',
 'KBI메탈.xlsx',
 'KB오토시스.xlsx',
 'KCC건설.xlsx',
 'KC그린홀딩스.xlsx',
 'KC코트렐.xlsx',
 'KD.

In [585]:
def concat_df(df, name, name_pos):
    col_list = []
    for i, col in enumerate(df.columns):
        try:
            int(col[:8])
            col_list.append(i)
        except:
            pass

    nm_df = df.iloc[:, name_pos]
    ye_df =  df.iloc[:, col_list]
    con_df = pd.concat([nm_df, ye_df], axis=1)
    filter_col = ['name']
    for col in con_df.columns[1:]:
        filter_col.append(col[:4])
    con_df.columns = filter_col
    con_df.set_index('name', inplace=True)
    con_df.index = [name]
    
   

    con_df = con_df.T.reset_index().drop_duplicates(subset=['index']).set_index('index').T
    con_df.columns.name = ''
    if '2009' in con_df.columns:
        con_df = con_df.drop('2009', axis=1)
    if '2021' in con_df.columns:
        con_df = con_df.drop('2021', axis=1)
        
    return con_df

In [588]:
df_li = []
count = 0
error_li = []
for i, f in enumerate(file_list):
   
        df = pd.read_excel(f"./fsdata/{f}", sheet_name=None)

        if df['Data_cis'].iloc[:, 1][0] == "label_ko":
            label_col = 1
        else:
            label_col = 2

        if 'Data_is' in df.keys():
            sheet_name = 'Data_is'
        else:
            sheet_name = 'Data_cis'

        # 자본금   
        df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].str.contains('자본금').fillna(False)] 
        if len(df1) > 1:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자본금']
        elif len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].str.contains('ifrs-full_IssuedCapital').fillna(False)] 
            if len(df1) == 0:
                df1 = df['Data_bs'][df['Data_bs'].iloc[:, 1].isin(['ifrs_EquityAttributableToOwnersOfParent']).fillna(False)] 
        if len(df1) == 0:
            df1 = df['Data_bs'][df['Data_bs'].iloc[:, label_col].isin(['납입자본'])] 

        # 자산총계
        df2 = df['Data_bs'][df['Data_bs'].iloc[:, label_col] == '자산총계']

        # 당기순이익
        df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기순이익').fillna(False)]
        if len(df3) > 1:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['연결당기순이익', '당기순이익(손실)', '당기순이익', '당기순손익', '당기순손실'])]
            if len(df3) == 0:
                df3 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].isin(['당기순이익(손실)'])]
        elif len(df3) == 0:
            df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].isin(['당기순손익', '당기순손실', '당기의 순이익', '계속영업이익(손실)'])]
            if len(df3) == 0:
                df3 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('당기순이익').fillna(False)]
            if len(df3) == 0:
                df3 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('당기손익').fillna(False)]
        if len(df3) > 1:
            df3 = df3[df3.isna().sum(axis = 1) == df3.isna().sum(axis = 1).min()]

        # 영업활동 현금흐름
        regex = r'[1-9]?[A-z]?[\{\}\[\]\/?.,;:|\)*~`!^\-_+<>@\#$%&\\\=\(\'\"]?\s?영업\s?활동?([가-힣]+)?(\s+[가-힣]+\s+)?\s?현금\s?흐름'
        df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains('ifrs-full_CashFlowsFromUsedInOperatingActivities').fillna(False)]                          

        if len(df4) == 0:
            df4 = df['Data_cf'][df['Data_cf'].iloc[:, label_col].str.contains(regex).fillna(False)]
            if len(df4) > 1:
                df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].isin(['ifrs_CashFlowsFromUsedInOperatingActivities']).fillna(False)]
                if len(df4) > 0:
                    df4 = df['Data_cf'][df['Data_cf'].iloc[:, 1].str.contains(r'영업\s활동\s현금\s흐름').fillna(False)]

        # 영업이익
        df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('dart_OperatingIncomeLoss').fillna(False)]
        if len(df5) == 0:
            df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]
            if len(df5) == 0:
                df5 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains(r'영업\s?이익').fillna(False)]
                if len(df5) == 0:
                    sheet_name = 'Data_cis'
                    df5 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'영업\s?이익').fillna(False)]

        # 매출액
        df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('ifrs-full_Revenue').fillna(False)] 
        if len(df6) == 0: # ID로 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        if len(df6) == 0: # Name으로 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('매출액').fillna(False)]
            if len(df6) == 0:
                df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].isin(['매출'])]

            if len(df6) > 1: 
                df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains(r'Ⅰ.매출액.*').fillna(False)]


        if len(df6) == 0: # name col다른 애들 col변경해서 잡기

            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].str.contains('매출액').fillna(False)]
            if len(df6) > 1:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['매출액'])]
            if len(df6) == 0:
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.매출'])]
            if len(df6) == 0:
                df6 = df['Data_cis'][df['Data_cis'].iloc[:, label_col].str.contains('매출액').fillna(False)]

        if len(df6) == 0:   # 매출액 없는녀석 영업수익 구하기
            df6 = df[sheet_name][df[sheet_name].iloc[:, label_col].str.contains('영업수익').fillna(False)]
            if len(df6) > 1: # name으로 isin
                df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
                if len(df6) > 1:
                    df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['I.영업수익', '영업수익'])]
        elif len(df6) > 1:
            df6 = df[sheet_name][df[sheet_name].iloc[:, 1].isin(['ifrs-full_Revenue', 'ifrs_Revenue'])]
        
        df1 = concat_df(df1, '자본금', label_col)
        df2 = concat_df(df2, '자산총계', label_col)
        df3 = concat_df(df3, '당기순이익', label_col)
        df4 = concat_df(df4, '영업활동현금흐름', label_col)
        df5 = concat_df(df5, '영업이익', label_col)
        df6 = concat_df(df6, '매출액', label_col)
        file_n = f.replace('.xlsx', '')
        if file_n in fi_df_dic.keys():
            fi_df = fi_df_dic[file_n]
            con_df = pd.concat([df1, df2, df3, df4, df5, df6, fi_df])
            con_df.to_csv(f'./data/{file_n}.csv')
        
        
        #print(df1.index.values, df2.index.values, df3.index.values, df4.index.values, df5.index.values, df6.index.values, i, f)
       # print(i, f, len(df1), len(df2), len(df3), len(df4), len(df5), len(df6))
#         if len(df1) != 1 or len(df2) != 1 or len(df3) != 1 or len(df4) != 1 or len(df5) != 1 or len(df6) != 1:
#             count += 1
#             error_li.append([i, f])
#             print('Error')
    
print(error_li)
print(count)

C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extract.
  return func(self, *args, **kwargs)
C:\Users\bitcamp\anaconda3\lib\site-packages\pandas\core\strings\accessor.py:101: UserWarning: This pattern has match groups. To actually get the groups, use str.extrac

[]
0


In [410]:
path = "data"
data_list = os.listdir(path)
data_list

['3S.csv',
 'AJ네트웍스.csv',
 'AK홀딩스.csv',
 'APS홀딩스.csv',
 'AP시스템.csv',
 'BGF.csv',
 'BGF리테일.csv',
 'BNK금융지주.csv',
 'BYC.csv',
 'CJ CGV.csv',
 'CJ ENM.csv',
 'CJ.csv',
 'CJ대한통운.csv',
 'CJ씨푸드.csv',
 'CJ제일제당.csv',
 'CJ프레시웨이.csv',
 'CMG제약.csv',
 'CS홀딩스.csv',
 'DB.csv',
 'DB하이텍.csv',
 'DMS.csv',
 'DRB동일.csv',
 'DSR.csv',
 'E1.csv',
 'EDGC.csv',
 'EG.csv',
 'EMW.csv',
 'ESR켄달스퀘어리츠.csv',
 'GH신소재.csv',
 'GS.csv',
 'GS건설.csv',
 'GS글로벌.csv',
 'GS리테일.csv',
 'HB테크놀러지.csv',
 'HDC.csv',
 'HDC현대EP.csv',
 'HDC현대산업개발.csv',
 'HMM.csv',
 'HRS.csv',
 'HSD엔진.csv',
 'iMBC.csv',
 'ISC.csv',
 'ITX-AI.csv',
 'JTC.csv',
 'JW생명과학.csv',
 'JW신약.csv',
 'JW중외제약.csv',
 'JW홀딩스.csv',
 'JYP Ent..csv',
 'KBI메탈.csv',
 'KB오토시스.csv',
 'KCC건설.csv',
 'KC그린홀딩스.csv',
 'KC코트렐.csv',
 'KD.csv',
 'KEC.csv',
 'KG ETS.csv',
 'KG동부제철.csv',
 'KG모빌리언스.csv',
 'KG이니시스.csv',
 'KG케미칼.csv',
 'KH바텍.csv',
 'KISCO홀딩스.csv',
 'KMH.csv',
 'KMH하이텍.csv',
 'KNN.csv',
 'KPX케미칼.csv',
 'KPX홀딩스.csv',
 'KR모터스.csv',
 'KSS해운.csv',
 'KTcs.csv',
 'LF.csv',
 'LG

In [414]:
for f in data_list[:1]:
    df = pd.read_csv(f"./data/{f}")

In [415]:
df

,Unnamed: 0,2021,2020,2019,2018,2017,2016,2015,2014,2013,2012,2011,2010
0,자본총계,2.283827e+10,22401255500,22386071500,21833121000,20987069000,20446134000,20446134000,20446134000,20446134000,19530576500,16317098000,14699059500
1,자산총계,5.461475e+10,49658374621,53329221212,49520999059,53706903697,57746389196,67359971696,71312012159,67660583293,50464379985,36432133064,28141189033
2,당기순이익,NaN,695334883,-1080358319,1411081065,-1432239774,-8803810083,-3403437816,-5599032927,-4760633476,701146938,1588455887,2752232738
3,영업활동현금흐름,NaN,871088551,1188555065,2854514902,4555587479,-2881262336,1919991836,350598217,1960176107,3312258405,488508829,-622678450
4,영업이익,NaN,-1408265950,467181073,1662145638,333310617,-2421224869,1176535923,-4157989321,-2540739501,1859291010,2844929574,3656104800
5,매출액,NaN,23477908383,22592015218,35751939484,31546170185,26486287918,23742421655,23210541648,28862985297,32236446153,31177983954,27394089729
6,종가,NaN,2165,2820,2215,1845,2785,2335,2395,4440,7160,25100,3270
7,시가총액,NaN,96997436315,126257443260,98336869770,77442284610,116897974330,95483445780,97936981860,181561669920,292026041600,980434940300,103393783380
8,상장주식수,NaN,44802511,44772143,44395878,41974138,41974138,40892268,40892268,40892268,40785760,39061153,31618894


In [418]:
path = "./data"
file_list = os.listdir(path)
file_list

['3S.csv',
 'AJ네트웍스.csv',
 'AK홀딩스.csv',
 'APS홀딩스.csv',
 'AP시스템.csv',
 'BGF.csv',
 'BGF리테일.csv',
 'BNK금융지주.csv',
 'BYC.csv',
 'CJ CGV.csv',
 'CJ ENM.csv',
 'CJ.csv',
 'CJ대한통운.csv',
 'CJ씨푸드.csv',
 'CJ제일제당.csv',
 'CJ프레시웨이.csv',
 'CMG제약.csv',
 'CS홀딩스.csv',
 'DB.csv',
 'DB하이텍.csv',
 'DMS.csv',
 'DRB동일.csv',
 'DSR.csv',
 'E1.csv',
 'EDGC.csv',
 'EG.csv',
 'EMW.csv',
 'ESR켄달스퀘어리츠.csv',
 'GH신소재.csv',
 'GS.csv',
 'GS건설.csv',
 'GS글로벌.csv',
 'GS리테일.csv',
 'HB테크놀러지.csv',
 'HDC.csv',
 'HDC현대EP.csv',
 'HDC현대산업개발.csv',
 'HMM.csv',
 'HRS.csv',
 'HSD엔진.csv',
 'iMBC.csv',
 'ISC.csv',
 'ITX-AI.csv',
 'JTC.csv',
 'JW생명과학.csv',
 'JW신약.csv',
 'JW중외제약.csv',
 'JW홀딩스.csv',
 'JYP Ent..csv',
 'KBI메탈.csv',
 'KB오토시스.csv',
 'KCC건설.csv',
 'KC그린홀딩스.csv',
 'KC코트렐.csv',
 'KD.csv',
 'KEC.csv',
 'KG ETS.csv',
 'KG동부제철.csv',
 'KG모빌리언스.csv',
 'KG이니시스.csv',
 'KG케미칼.csv',
 'KH바텍.csv',
 'KISCO홀딩스.csv',
 'KMH.csv',
 'KMH하이텍.csv',
 'KNN.csv',
 'KPX케미칼.csv',
 'KPX홀딩스.csv',
 'KR모터스.csv',
 'KSS해운.csv',
 'KTcs.csv',
 'LF.csv',
 'LG

In [456]:
for f in file_list:
    df = pd.read_csv(f"./data/{f}", index_col=0)
    if df_index in reg_list:
        
    

3S.csv Check
3S.csv Check
3S.csv Check
3S.csv Check
3S.csv Check
3S.csv Check
3S.csv Check
3S.csv Check
3S.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AJ네트웍스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
AK홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
APS홀딩스.csv Check
AP시스템.csv Check
AP시스템.csv Check
AP시스템.csv Check
AP시스템.csv Check
AP시스템.csv Check
AP시스템.csv Check
AP시스템.csv Check
AP시스템.csv Check
AP시스템.csv Check
BGF.csv Check
BGF.csv Check
BGF.csv Check
BGF.csv Check
BGF.csv Check
BGF.csv Check
BGF.csv Check
BGF.csv Check
BGF.csv Check
BGF리테일.csv Check
BGF리테일.csv Check
BGF리테일.csv Error
BGF리테일.csv Check
BGF리테일.csv Error
BGF리테일.csv Check
BGF리테일.csv Check
BGF리테일.csv Check
BGF리테일.csv Check
BNK금융지주.cs

KG이니시스.csv Check
KG이니시스.csv Check
KG이니시스.csv Check
KG이니시스.csv Check
KG이니시스.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KG케미칼.csv Check
KH바텍.csv Check
KH바텍.csv Check
KH바텍.csv Check
KH바텍.csv Check
KH바텍.csv Check
KH바텍.csv Check
KH바텍.csv Check
KH바텍.csv Check
KH바텍.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KISCO홀딩스.csv Check
KMH.csv Check
KMH.csv Check
KMH.csv Check
KMH.csv Check
KMH.csv Check
KMH.csv Check
KMH.csv Check
KMH.csv Check
KMH.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KMH하이텍.csv Check
KNN.csv Check
KNN.csv Check
KNN.csv Check
KNN.csv Check
KNN.csv Check
KNN.csv Check
KNN.csv Check
KNN.csv Check
KNN.csv Check
KPX케미칼.csv Check
KPX케미칼.csv Check
KPX케미칼.csv Check
KPX케미칼.cs

SNK.csv Check
SNK.csv Check
SNK.csv Check
SNK.csv Check
SNK.csv Check
SNK.csv Check
SNK.csv Check
SNK.csv Check
SNK.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
SPC삼립.csv Check
STX.csv Check
STX.csv Check
STX.csv Check
STX.csv Check
STX.csv Check
STX.csv Check
STX.csv Check
STX.csv Check
STX.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX엔진.csv Check
STX중공업.csv Check
STX중공업.csv Check
STX중공업.csv Check
STX중공업.csv Check
STX중공업.csv Check
STX중공업.csv Check
STX중공업.csv Check
STX중공업.csv Check
STX중공업.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
TCC스틸.csv Check
THE E&M.csv Check
THE E&M.csv Check
THE E&M.csv Check
THE E&M.csv Check
THE E&M.csv Check
THE E&M.csv Check
THE E&M.csv Check
THE E&M.csv Check
THE E&M.csv Check
T

나우IB.csv Check
나우IB.csv Check
나우IB.csv Check
나우IB.csv Check
나우IB.csv Check
나우IB.csv Check
나우IB.csv Check
나우IB.csv Check
나우IB.csv Check
나우코스.csv Check
나우코스.csv Check
나우코스.csv Check
나우코스.csv Check
나우코스.csv Check
나우코스.csv Check
나우코스.csv Check
나우코스.csv Check
나우코스.csv Check
네패스.csv Check
네패스.csv Check
네패스.csv Check
네패스.csv Check
네패스.csv Check
네패스.csv Check
네패스.csv Check
네패스.csv Check
네패스.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥센타이어.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트BT.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트사이언스.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스트아이.csv Check
넥스틴.csv Ch

대한과학.csv Check
대한과학.csv Check
대한과학.csv Check
대한과학.csv Check
대한과학.csv Check
대한과학.csv Check
대한과학.csv Check
대한과학.csv Check
대한과학.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한광통신.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한그린파워.csv Check
대한방직.csv Check
대한방직.csv Check
대한방직.csv Check
대한방직.csv Check
대한방직.csv Check
대한방직.csv Check
대한방직.csv Check
대한방직.csv Check
대한방직.csv Check
대한유화.csv Check
대한유화.csv Check
대한유화.csv Check
대한유화.csv Check
대한유화.csv Check
대한유화.csv Check
대한유화.csv Check
대한유화.csv Check
대한유화.csv Check
대한전선.csv Check
대한전선.csv Check
대한전선.csv Check
대한전선.csv Check
대한전선.csv Check
대한전선.csv Check
대한전선.csv Check
대한전선.csv Check
대한전선.csv Check
대한제강.csv Check
대한제강.csv Check
대한제강.csv Check
대한제강.csv Check
대한제강.csv Check
대한제강.csv Check
대한제강.csv Check
대한제강.csv Check
대한제강.csv Check
대한제당.csv Check
대한제당.csv Chec

...

코리아에스이.csv Check
코리아에스이.csv Check
코리아에스이.csv Check
코리아에스이.csv Check
코리아에스이.csv Check
코리아에스이.csv Check
코리아에스이.csv Check
코리아에스이.csv Check
코리아에스이.csv Check
코맥스.csv Check
코맥스.csv Check
코맥스.csv Check
코맥스.csv Check
코맥스.csv Check
코맥스.csv Check
코맥스.csv Check
코맥스.csv Check
코맥스.csv Check
코메론.csv Check
코메론.csv Check
코메론.csv Check
코메론.csv Check
코메론.csv Check
코메론.csv Check
코메론.csv Check
코메론.csv Check
코메론.csv Check
코미코.csv Check
코미코.csv Check
코미코.csv Check
코미코.csv Check
코미코.csv Check
코미코.csv Check
코미코.csv Check
코미코.csv Check
코미코.csv Check
코미팜.csv Check
코미팜.csv Check
코미팜.csv Check
코미팜.csv Check
코미팜.csv Check
코미팜.csv Check
코미팜.csv Check
코미팜.csv Check
코미팜.csv Check
코스맥스.csv Check
코스맥스.csv Check
코스맥스.csv Error
코스맥스.csv Check
코스맥스.csv Check
코스맥스.csv Check
코스맥스.csv Check
코스맥스.csv Check
코스맥스.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스비티아이.csv Check
코스맥스엔비티.csv Check
코스맥스엔비티.csv Check
코

토니모리.csv Check
토니모리.csv Check
토니모리.csv Check
토니모리.csv Check
토니모리.csv Check
토비스.csv Check
토비스.csv Check
토비스.csv Check
토비스.csv Check
토비스.csv Check
토비스.csv Check
토비스.csv Check
토비스.csv Check
토비스.csv Check
톱텍.csv Check
톱텍.csv Check
톱텍.csv Check
톱텍.csv Check
톱텍.csv Check
톱텍.csv Check
톱텍.csv Check
톱텍.csv Check
톱텍.csv Check
투비소프트.csv Check
투비소프트.csv Check
투비소프트.csv Check
투비소프트.csv Check
투비소프트.csv Check
투비소프트.csv Check
투비소프트.csv Check
투비소프트.csv Check
투비소프트.csv Check
트루윈.csv Check
트루윈.csv Check
트루윈.csv Check
트루윈.csv Check
트루윈.csv Check
트루윈.csv Check
트루윈.csv Check
트루윈.csv Check
트루윈.csv Check
특수건설.csv Check
특수건설.csv Check
특수건설.csv Check
특수건설.csv Check
특수건설.csv Check
특수건설.csv Check
특수건설.csv Check
특수건설.csv Check
특수건설.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티로보틱스.csv Check
티사이언티픽.csv Check
티사이언티픽.csv Check
티사이언티픽.csv Check
티사이언티픽.csv Check
티사이언티픽.csv Check
티사이언티픽.csv Check
티사이언티픽.csv Check
티사이언티픽.csv Che

프리엠스.csv Check
프리엠스.csv Check
프리엠스.csv Check
프리엠스.csv Check
프리엠스.csv Check
프리엠스.csv Check
프리엠스.csv Check
프리엠스.csv Check
프리엠스.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플랜티넷.csv Check
플레이위드.csv Check
플레이위드.csv Check
플레이위드.csv Check
플레이위드.csv Check
플레이위드.csv Check
플레이위드.csv Check
플레이위드.csv Check
플레이위드.csv Check
플레이위드.csv Check
플리토.csv Check
플리토.csv Check
플리토.csv Check
플리토.csv Check
플리토.csv Check
플리토.csv Check
플리토.csv Check
플리토.csv Check
플리토.csv Check
피노텍.csv Check
피노텍.csv Check
피노텍.csv Check
피노텍.csv Check
피노텍.csv Check
피노텍.csv Check
피노텍.csv Check
피노텍.csv Check
피노텍.csv Check
피에스엠씨.csv Check
피에스엠씨.csv Check
피에스엠씨.csv Error
피에스엠씨.csv Check
피에스엠씨.csv Error
피에스엠씨.csv Check
피에스엠씨.csv Check
피에스엠씨.csv Check
피에스엠씨.csv Check
피에스케이.csv Check
피에스케이.csv Check
피에스케이.csv Check
피에스케이.csv Check
피에스케이.csv Check
피에스케이.csv Check
피에스케이.csv Check
피에스케이.csv Check
피에스케이.csv Check
픽셀플러스.csv Check
픽셀플러스.csv Check
픽셀플러스.csv Chec

한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔인티큐브.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔제지.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔케미칼.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔테크닉스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홀딩스.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한솔홈데코.csv Check
한스바이오메드.csv Error
한스바이오메드.csv Check
한스바이오메드.csv Error
한스바이오메드.csv Check
한스바이오메드.csv Error
한스바이오메드.csv Error
한스바이오메드.csv Check
한

현대홈쇼핑.csv Check
현대홈쇼핑.csv Check
현대홈쇼핑.csv Check
현대홈쇼핑.csv Check
현대홈쇼핑.csv Check
현우산업.csv Check
현우산업.csv Check
현우산업.csv Check
현우산업.csv Check
현우산업.csv Check
현우산업.csv Check
현우산업.csv Check
현우산업.csv Check
현우산업.csv Check
현진소재.csv Check
현진소재.csv Check
현진소재.csv Check
현진소재.csv Check
현진소재.csv Check
현진소재.csv Check
현진소재.csv Check
현진소재.csv Check
현진소재.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지I&C.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
형지엘리트.csv Check
혜인.csv Check
혜인.csv Check
혜인.csv Check
혜인.csv Check
혜인.csv Check
혜인.csv Check
혜인.csv Check
혜인.csv Check
혜인.csv Check
호전실업.csv Check
호전실업.csv Check
호전실업.csv Check
호전실업.csv Check
호전실업.csv Check
호전실업.csv Check
호전실업.csv Check
호전실업.csv Check
호전실업.csv Check
호텔신라.csv Check
호텔신라.csv Check
호텔신라.csv Check
호텔신라.csv Check
호텔신라.csv Check
호텔신라.csv Check
호텔신라.csv Check
호텔신라.

In [475]:
reg_list = ['자본총계', '자산총계', '당기순이익', '영업활동현금흐름', '영업이익', '매출액', '종가', '시가총액',
       '상장주식수']

In [476]:
reg_list

['자본총계', '자산총계', '당기순이익', '영업활동현금흐름', '영업이익', '매출액', '종가', '시가총액', '상장주식수']

In [477]:
con_df.index

Index(['자본총계', '자산총계', '당기순이익', '영업활동현금흐름', '영업이익', '매출액', '종가', '시가총액',
       '상장주식수'],
      dtype='object')

In [479]:
con_df.index

Index(['자본총계', '자산총계', '당기순이익', '영업활동현금흐름', '영업이익', '매출액', '종가', '시가총액',
       '상장주식수'],
      dtype='object')

In [480]:
reg_list

['자본총계', '자산총계', '당기순이익', '영업활동현금흐름', '영업이익', '매출액', '종가', '시가총액', '상장주식수']